In [1]:
import speech_recognition as sr

class SpeechToText:
    def __init__(self, device_index=1):
        self.r = sr.Recognizer()  # Recognizer nesnesi oluşturma
        self.microphone = sr.Microphone(device_index=device_index) # Mikrofon nesnesi oluşturma
        self.calibrate_microphone() # Mikrofonu kalibre etme

    def calibrate_microphone(self):
        with self.microphone as source:
            print("Mikrofon kalibre ediliyor. Lütfen sessiz olunuz")
            self.r.adjust_for_ambient_noise(source, duration=2) # Mikrofonu kalibre etme
            self.r.energy_threshold += 75 # Mikrofon kalibrasyonu için eşik değerini ayarlama
            self.calculated_energy_threshold = self.r.energy_threshold # Eşik değerini kaydetme

    def speech_to_text(self, audio):
        if not isinstance(audio, sr.AudioData): # Dosyanın ses dosyası olduğunu kontrol etme
            print("Dosya bir ses dosyası değil.")
            return

        try:
            return self.r.recognize_google(audio, language='tr-TR') # Ses dosyasını metne çevirme
        except sr.UnknownValueError:
            return "Anlaşılamayan ses."
        except sr.RequestError:
            return "Zaman aşımı"

    def listen_command(self):
        audio = self.record_audio() # Ses kaydı alma
        if audio: # Ses kaydı alınmışsa
            text = self.speech_to_text(audio).lower() # Ses kaydını metne çevirme
            return self.process_text_command(text) # Metni döndürme
        else:
            return False

    def record_audio(self):
        self.r.energy_threshold = self.calculated_energy_threshold # Eşik değerini ayarlama
        self.r.dynamic_energy_threshold = False # Eşik değerinin dinamik olmamasını sağlama
        try:
            print("Sesli komut bekleniyor.")
            with self.microphone as source:
                return self.r.listen(source, 10, 1) # Ses kaydı alma
        except sr.WaitTimeoutError:
            return False

    def process_text_command(self, text):
        if "yukarı" in text: # Metin içerisinde yukarı kelimesi varsa
            return 1
        elif "aşağı" in text: # Metin içerisinde aşağı kelimesi varsa
            return 2
        elif "sol" in text: # Metin içerisinde sol kelimesi varsa
            return 3
        elif any(key in text for key in ["sağ", "saha"]): # Metin içerisinde sağ veya saha kelimelerinden biri varsa
            return 4
        else: # Yukarı, aşağı, sol veya sağ kelimelerinden biri yoksa
            print(text) 
            return False

In [2]:
import pygame
import sys
import random
import threading

# Ekran Boyutları
WIDTH, HEIGHT = 800, 600

# Yılan Boyutları
SNAKE_SIZE = 3
SNAKE_BLOCK_SIZE = 20
GAME_SPEED = 3

# Renkler
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
GREY = (128, 128, 128)

# Yönler
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

class SnakeGame:
    def __init__(self):
        pygame.init() # Pygame başlatma
        self.init_display() # Ekranı başlatma

        self.mic = SpeechToText() # Sesi metne dönüştürme nesnesi oluşturma

        self.snake = [(WIDTH // 2, HEIGHT // 2)] # Yılanın başlangıç pozisyonu
        self.direction = RIGHT
        self.food = self.generate_food() # İlk yemi oluşturma

    def init_display(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT)) # Ekranı oluşturma
        pygame.display.set_caption("Snake Game") # Ekran başlığını ayarlama
        self.clock = pygame.time.Clock() # Oyun hızını ayarlama

    def generate_food(self): # Yemi rastgele konuma yerleştirme
        food_x = random.randrange(0, WIDTH // SNAKE_BLOCK_SIZE) * SNAKE_BLOCK_SIZE
        food_y = random.randrange(0, HEIGHT // SNAKE_BLOCK_SIZE) * SNAKE_BLOCK_SIZE
        return food_x, food_y # Yemin konumunu döndürme

    def run(self):
        voice_thread = threading.Thread(target=self.listen_voice_commands) # Ses komutlarını dinleme işlemini başlatma
        voice_thread.start()

        while True:
            self.handle_events() # Olayları kontrol etme
            self.move() # Yılanı hareket ettirme
            self.check_collision() # Çarpışmayı kontrol etme
            self.check_food() # Yemi kontrol etme
            self.adjust_snake_position() # Ekranın dışına çıkan yılanı düzeltme
            self.update_display() # Ekranı güncelleme

    def listen_voice_commands(self):
        while True:
            direction = self.mic.listen_command() # Sesli komutlardan yön alma
            self.update_direction(direction) # Yönü güncelleme

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.quit_game()

    def move(self):
        head = self.snake[0] # Yılanın başını al
        new_head = (head[0] + self.direction[0] * SNAKE_BLOCK_SIZE, head[1] + self.direction[1] * SNAKE_BLOCK_SIZE) # Yılanın yeni baş pozisyonunu hesapla
        self.snake.insert(0, new_head) # Yılanın başını yeni pozisyona ekle
        if len(self.snake) > SNAKE_SIZE: # Yılanın yeni boyutu, yılan boyutundan büyükse
            self.snake.pop() # Yılanın kuyruğunu kes

    def adjust_snake_position(self): 
        head = self.snake[0] # Yılanın başını al
        if head[0] < 0 or head[0] >= WIDTH or head[1] < 0 or head[1] >= HEIGHT: # Yılanın başı ekranın dışına çıkmışsa
            self.wrap_around_screen() # Yılanı ekranın içine al

    def wrap_around_screen(self):
        head = self.snake[0] # Yılanın başını al
        self.snake[0] = (head[0] % WIDTH, head[1] % HEIGHT) # Yılanın başını ekranın karşısına al

    def check_collision(self): 
        head = self.snake[0] # Yılanın başını al
        if head in self.snake[1:]: # Yılanın başı yılana çarpmışsa
            self.quit_game() # Oyunu bitir

    def check_food(self):
        head = self.snake[0] # Yılanın başını al
        if head == self.food: # Yılanın başı yeme çarpmışsa
            self.food = self.generate_food() # Yeni yem oluştur
            self.snake.append((0, 0)) # Yılanı büyüt

    def update_direction(self, direction):
        if direction == 1 and self.direction != DOWN: # Komut 1'se ve yılan aşağı gitmiyorsa
            self.direction = UP
        elif direction == 2 and self.direction != UP: # Komut 2'yse ve yılan yukarı gitmiyorsa
            self.direction = DOWN
        elif direction == 3 and self.direction != RIGHT: # Komut 3'se ve yılan sağa gitmiyorsa
            self.direction = LEFT
        elif direction == 4 and self.direction != LEFT: # Komut 4'se ve yılan sola gitmiyorsa
            self.direction = RIGHT

    def update_display(self):
        self.screen.fill(GREY) # Ekranı gri renge boyama
        self.draw_snake() # Yılanı çizme
        self.draw_food() # Yemi çizme
        pygame.display.flip() # Ekranı güncelleme
        self.clock.tick(GAME_SPEED) # Oyun hızını ayarlama

    def draw_snake(self):
        for segment in self.snake: 
            pygame.draw.rect(self.screen, GREEN, (segment[0], segment[1], SNAKE_BLOCK_SIZE, SNAKE_BLOCK_SIZE)) # Yılanı çizme

    def draw_food(self):
        pygame.draw.rect(self.screen, RED, (self.food[0], self.food[1], SNAKE_BLOCK_SIZE, SNAKE_BLOCK_SIZE)) # Yemi çizme

    def quit_game(self):
        pygame.quit()
        sys.exit()

if __name__ == "__main__":
    game = SnakeGame()
    game.run()

pygame 2.1.3 (SDL 2.0.22, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
Mikrofon kalibre ediliyor. Lütfen sessiz olunuz
Sesli komut bekleniyor.
Sesli komut bekleniyor.
Sesli komut bekleniyor.


SystemExit: 

C:\Users\aomin\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


anlaşılamayan ses.
Sesli komut bekleniyor.
anlaşılamayan ses.
Sesli komut bekleniyor.
Sesli komut bekleniyor.
Sesli komut bekleniyor.
anlaşılamayan ses.
Sesli komut bekleniyor.
Sesli komut bekleniyor.
Sesli komut bekleniyor.
Sesli komut bekleniyor.
